In [1]:
# Cell 1: Import necessary libraries
import os
import glob
import numpy as np
import tifffile as tiff
from concurrent.futures import ProcessPoolExecutor, as_completed

print("Libraries imported. Ready to process on CPUs in parallel!")

Libraries imported. Ready to process on CPUs in parallel!


In [2]:
# Cell 2: Define your directories
# Set the input directory that contains your multi-page TIFF files
input_dir = '/v-data4/kbarber/mbo_lbm/03_13_2025/mk301/red/'  # CHANGE THIS to your input folder path

# Define the base directory for outputs (each imaging plane will have its own folder)
output_base_dir = '/v-data4/kbarber/2025/mbo_lbm_proccessed/mk301_03_13_2025/red_dont_use_test/'  # CHANGE THIS to your desired output folder path

# Create the output base directory if it doesn't exist
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

print("Input directory:", input_dir)
print("Output base directory:", output_base_dir)


Input directory: /v-data4/kbarber/mbo_lbm/03_13_2025/mk301/red/
Output base directory: /v-data4/kbarber/2025/mbo_lbm_proccessed/mk301_03_13_2025/red_dont_use_test/


In [ ]:
# List of plane numbers (1-indexed) for which to shift the odd rows.
shift_plane_numbers = []

# Expected dimensions for each frame before reshaping.
expected_height, expected_width = 912, 224
# After reshaping, each frame will be (456, 448) because:
# top half = rows 0:456, bottom half = rows 456:912, concatenated along width.
# ============================

def process_tif_file(tif_file):
    """
    Process a single 4D TIFF file (shape: (num_timepoints, 14, 912, 224)):
      - For each plane, extract the time series.
      - Reshape each frame from (912, 224) to (456, 448) by splitting and concatenating.
      - For specified planes (per shift_plane_numbers), shift odd-indexed rows left by 1 pixel.
      - Save each processed plane stack to its subfolder in output_base_dir.
    """
    try:
        # Load the TIFF file
        stack = tiff.imread(tif_file)

        # Check that the file has 4 dimensions
        if stack.ndim != 4:
            print(f"File {tif_file} does not have 4 dimensions (found shape {stack.shape}), skipping.")
            return

        num_timepoints, num_planes, height, width = stack.shape
        if height != expected_height or width != expected_width:
            print(f"Warning: File {tif_file} has unexpected frame shape {height}x{width} (expected {expected_height}x{expected_width}). Skipping.")
            return

        base_name = os.path.splitext(os.path.basename(tif_file))[0]

        # Process each plane (0-indexed; will be saved as plane_1, plane_2, etc.)
        for plane in range(num_planes):
            # Create output directory for this plane if it doesn't exist
            plane_dir = os.path.join(output_base_dir, f'plane_{plane+1}')
            if not os.path.exists(plane_dir):
                os.makedirs(plane_dir)

            # Extract the plane's time series (shape: (num_timepoints, 912, 224))
            plane_stack = stack[:, plane, :, :]

            # Reshape each frame:
            #   - top_half: rows 0 to 455 (456 rows)
            #   - bottom_half: rows 456 to 911 (456 rows)
            #   - Concatenate horizontally to produce (456, 448) frames.
            top_half = plane_stack[:, :456, :]    # shape: (num_timepoints, 456, 224)
            bottom_half = plane_stack[:, 456:, :]   # shape: (num_timepoints, 456, 224)
            reshaped_stack = np.concatenate([top_half, bottom_half], axis=2)  # shape: (num_timepoints, 456, 448)

            # If this plane is one of the ones to be shifted, apply the shift:
            if (plane + 1) in shift_plane_numbers:
                # Make a copy for shifting
                shifted_stack = np.copy(reshaped_stack)
                # Identify odd-indexed rows (rows 1, 3, 5, ...)
                odd_row_indices = np.arange(1, reshaped_stack.shape[1], 2)
                # For each frame, shift the odd rows left by 1 pixel:
                shifted_stack[:, odd_row_indices, :-1] = reshaped_stack[:, odd_row_indices, 1:]
                shifted_stack[:, odd_row_indices, -1] = 0
                processed_stack = shifted_stack
            else:
                processed_stack = reshaped_stack

            # Save the processed stack as a multi-page TIFF.
            output_file = os.path.join(plane_dir, f'{base_name}_plane{plane+1}.tif')
            tiff.imwrite(output_file, processed_stack)

        print(f"Processed: {tif_file}")
    except Exception as e:
        print(f"Error processing {tif_file}: {e}")

# Get the list of TIFF files from the input directory (adjust the extension if needed)
tif_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
print(f"Found {len(tif_files)} TIFF file(s) to process.")

# Process the files in parallel using ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    futures = [executor.submit(process_tif_file, tif_file) for tif_file in tif_files]
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as exc:
            print(f"A processing task generated an exception: {exc}")

print("\nParallel processing complete!")

Found 0 TIFF file(s) to process.

Parallel processing complete!


In [ ]:
# Cell: Generate Mean Projection JPEGs with Automatic Level Adjustment

import os
import glob
import numpy as np
import tifffile as tiff
from PIL import Image

# Try to import scikit-image's exposure module for contrast adjustment.
try:
    from skimage import exposure
    def adjust_intensity(image):
        # Compute the 2nd and 98th percentiles.
        p2, p98 = np.percentile(image, (2, 99))
        # Rescale intensities so that [p2, p98] maps to [0, 255].
        return exposure.rescale_intensity(image, in_range=(p2, p98), out_range=(0, 255))
except ModuleNotFoundError:
    print("scikit-image not found. Using custom intensity adjustment function.")
    def adjust_intensity(image):
        # Compute the 2nd and 98th percentiles.
        p2, p98 = np.percentile(image, (2, 99))
        # Clip and rescale manually.
        image_clipped = np.clip(image, p2, p98)
        return (image_clipped - p2) / (p98 - p2) * 255

# Input: the folder that contains your plane folders (e.g., "plane_1", "plane_2", ..., "plane_14")
processed_folder = '/v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/red/'  # CHANGE THIS: folder containing the plane folders

# Output: the folder where the mean projection JPEG images will be saved
jpeg_output_folder = '/v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/mean_projections_red'  # CHANGE THIS as needed

# Create the output folder if it doesn't exist.
if not os.path.exists(jpeg_output_folder):
    os.makedirs(jpeg_output_folder)

# Get a sorted list of plane folders.
plane_folders = sorted(glob.glob(os.path.join(processed_folder, 'plane_*')))
print(f"Found {len(plane_folders)} plane folder(s).")

# Process each plane folder.
for plane_folder in plane_folders:
    plane_name = os.path.basename(plane_folder)  # e.g., "plane_1"

    # Get the list of TIFF files in this folder (we'll use the first one).
    tiff_files = sorted(glob.glob(os.path.join(plane_folder, '*.tif')))
    if len(tiff_files) == 0:
        print(f"No TIFF files found in {plane_folder}, skipping.")
        continue

    first_file = tiff_files[0]
    print(f"Processing {first_file} for {plane_name}...")

    # Load the TIFF stack. Expected shape: (num_frames, height, width)
    stack = tiff.imread(first_file)

    # Compute the mean projection across the time axis.
    mean_image = np.mean(stack, axis=0)

    # Adjust the contrast based on percentiles.
    mean_image_adjusted = adjust_intensity(mean_image)

    # Ensure the image is in uint8 format for saving as JPEG.
    mean_image_uint8 = np.clip(mean_image_adjusted, 0, 255).astype(np.uint8)

    # Save the resulting image as a JPEG.
    output_jpeg = os.path.join(jpeg_output_folder, f'{plane_name}_mean_projection.jpg')
    im = Image.fromarray(mean_image_uint8)
    im.save(output_jpeg, quality=95)

    print(f"Saved mean projection (adjusted) for {plane_name} to {output_jpeg}")

print("Mean projection image generation with adjusted levels complete!")



scikit-image not found. Using custom intensity adjustment function.
Found 14 plane folder(s).
Processing /v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/red/plane_1/mk303_2_5_25_2roi_17p07hz_224x448px_2umpx_170mw_red_after_00001_plane1.tif for plane_1...
Saved mean projection (adjusted) for plane_1 to /v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/mean_projections_red/plane_1_mean_projection.jpg
Processing /v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/red/plane_10/mk303_2_5_25_2roi_17p07hz_224x448px_2umpx_170mw_red_after_00001_plane10.tif for plane_10...
Saved mean projection (adjusted) for plane_10 to /v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/mean_projections_red/plane_10_mean_projection.jpg
Processing /v-data4/kbarber/2025/mbo_lbm_proccessed/mk303_02_10_2025/red/plane_11/mk303_2_5_25_2roi_17p07hz_224x448px_2umpx_170mw_red_after_00001_plane11.tif for plane_11...
Saved mean projection (adjusted) for plane_11 to /v-data4/kbarber/2025/mbo_